## Data import

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os, shutil
!ls "/content/gdrive/My Drive/"

 8-2.mov
 Bayesian
 bigdata
 bq-results-20190920-004228-5xi034liest1
 bq-results-20190920-012138-er5eg3s9lxkj
 bq-results-20190920-012622-6g3qu1tkjun8
 bq-results-20191112-190906-w6n93g514345
 CAMELYON16
 checkpoints
'Chu Yu-resume.pdf'
'Colab Notebooks'
'data science final scratch.gdoc'
'Download eTranscripts ordered from your school (Windows) (1).gdoc'
 drug.csv
'epi assignment 5.gdoc'
 measure.csv
 my_model.h5
 observation.csv
'Phar stats final SAP.gdoc'
 scudata
 sculpture


In [0]:
drug_df_o = pd.read_csv('/content/gdrive/My Drive/drug.csv')

In [0]:
drug_df_o.head(5)

,person_id,visit_occurrence_id,year_of_birth,gender_source_value,ethnicity_concept_id,person_source_value,month_of_birth,day_of_birth,ethnicity_source_value,location_id,race_concept_id,gender_concept_id,birth_datetime,race_source_value,death_date,death_datetime,death_type_concept_id,cause_source_concept_id,provider_id,visit_start_date,care_site_id,visit_source_value,visit_end_date,visit_start_datetime,visit_concept_id,visit_type_concept_id,visit_end_datetime,drug_exposure_start_date,drug_source_value,quantity,days_supply,drug_source_concept_id,drug_exposure_id,drug_exposure_start_datetime,verbatim_end_date,drug_concept_id
0,5,4213630.0,1935,1,38003564,000308435E3E5B76,9,1,1,8,8557.0,8507,1935-09-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,68338.0,2009-01-09T00:00:00.000Z,2870.0,5.421323e+14,2009-01-09T00:00:00.000Z,2009-01-09T00:00:00.000Z,9202.0,44818517.0,2009-01-09T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41,1656926.0,1941,2,38003564,001F988D97A7759B,2,1,1,59,8527.0,8532,1941-02-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,179231.0,2008-10-09T00:00:00.000Z,42872.0,8.874434e+14,2008-10-09T00:00:00.000Z,2008-10-09T00:00:00.000Z,0.0,44818517.0,2008-10-09T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62,3963163.0,1943,1,38003564,002B6203FF086ABA,9,1,1,35,8516.0,8507,1943-09-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,189926.0,2008-06-30T00:00:00.000Z,86906.0,8.872734e+14,2008-06-30T00:00:00.000Z,2008-06-30T00:00:00.000Z,0.0,44818517.0,2008-06-30T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103,58348.0,1935,1,38003564,0050CF9059ABFF59,6,1,1,134,8657.0,8507,1935-06-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,42069.0,2009-07-25T00:00:00.000Z,5573.0,8.874534e+14,2009-07-25T00:00:00.000Z,2009-07-25T00:00:00.000Z,0.0,44818517.0,2009-07-25T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,125,1886636.0,1926,1,38003564,0064148837D38095,5,1,1,154,8657.0,8507,1926-05-01T00:00:00.000Z,1,NaN,NaN,NaN,NaN,54829.0,2009-01-13T00:00:00.000Z,30634.0,8.870134e+14,2009-01-13T00:00:00.000Z,2009-01-13T00:00:00.000Z,0.0,44818517.0,2009-01-13T00:00:00.000Z,2008-01-27T00:00:00.000Z,58016097590,10.0,0.0,44902911.0,2826933.0,2008-01-27T00:00:00.000Z,2015-10-13T00:00:00.000Z,1742284.0


In [0]:
drug_df = drug_df_o

In [0]:
#drug_df = drug_df.apply(lambda x:x.fillna(x.value_counts().index[0]))
drug_df = drug_df[drug_df['visit_start_datetime'] == drug_df.groupby('person_id')['visit_start_datetime'].transform('max')]

In [0]:
table1 = drug_df[['person_id', 'drug_exposure_id','quantity']]
table1[['person_id','drug_exposure_id']] = table1[['person_id','drug_exposure_id']].astype(str)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [0]:
table1.groupby('person_id')['drug_exposure_id'].apply(len).head(20)

person_id
0         1
10        1
100       5
10000     1
100000    2
100001    1
100003    1
100005    6
100007    1
100008    1
10001     1
100010    1
100011    5
100012    1
100013    1
100014    1
100015    1
100017    9
100019    1
10002     1
Name: drug_exposure_id, dtype: int64

## data preprocessing

In [0]:
persondrop = ['person_source_value','ethnicity_source_value', 'gender_source_value','location_id', 'race_source_value', 'care_site_id']
visitdrop = ['visit_source_value','visit_start_date','visit_end_date','visit_end_datetime']
drugdrop = ['drug_source_value', 'drug_source_concept_id', 'drug_exposure_start_date']

In [0]:
drug_df=drug_df.drop(columns=persondrop)
drug_df=drug_df.drop(columns=visitdrop)
drug_df=drug_df.drop(columns=drugdrop)

In [0]:
drug_df.loc[pd.isnull(drug_df['death_date']) , 'death'] = 0
drug_df.loc[pd.isnull(drug_df['death_date'])==False , 'death'] = 1

In [0]:
deathdrop=['death_date','death_datetime','death_type_concept_id','cause_source_concept_id']
drug_df=drug_df.drop(columns=deathdrop)

In [0]:
drug_df.head(5)

,person_id,visit_occurrence_id,year_of_birth,ethnicity_concept_id,month_of_birth,day_of_birth,race_concept_id,gender_concept_id,birth_datetime,provider_id,visit_start_datetime,visit_concept_id,visit_type_concept_id,quantity,days_supply,drug_exposure_id,drug_exposure_start_datetime,verbatim_end_date,drug_concept_id,death
4,125,1886636.0,1926,38003564,5,1,8657.0,8507,1926-05-01T00:00:00.000Z,54829.0,2009-01-13T00:00:00.000Z,0.0,44818517.0,10.0,0.0,2826933.0,2008-01-27T00:00:00.000Z,2015-10-13T00:00:00.000Z,1742284.0,0.0
5,190,5040769.0,1927,38003564,12,1,8516.0,8507,1927-12-01T00:00:00.000Z,117478.0,2009-04-20T00:00:00.000Z,0.0,44818517.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,225,2530146.0,1938,38003564,8,1,8657.0,8532,1938-08-01T00:00:00.000Z,221228.0,2009-10-29T00:00:00.000Z,0.0,44818517.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
19,512,2141915.0,1939,38003564,1,1,8527.0,8532,1939-01-01T00:00:00.000Z,238202.0,2009-07-27T00:00:00.000Z,0.0,44818517.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
21,573,989410.0,1936,38003564,3,1,8527.0,8532,1936-03-01T00:00:00.000Z,224.0,2009-11-16T00:00:00.000Z,0.0,44818517.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [0]:
len(drug_df)

199967

In [0]:
len(drug_df.person_id.unique)

In [0]:
## change the types of columns
categorical= ['person_id','visit_occurrence_id','ethnicity_concept_id','race_concept_id','gender_concept_id',
             'visit_concept_id','visit_type_concept_id','drug_exposure_id','drug_concept_id'] ## 9 objects
for col in categorical:
    drug_df[col] = drug_df[col].astype('object')
#drug_df['death'] = drug_df['death'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
import datetime
drug_df['visit_start_datetime'] = drug_df['visit_start_datetime'].map(lambda x: datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.000Z"))
drug_df['birth_datetime'] = drug_df['birth_datetime'].map(lambda x: datetime.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.000Z"))

In [0]:
drug_df['age'] = drug_df['visit_start_datetime']-drug_df['birth_datetime']
drug_df['age'] = drug_df['age'].dt.days.astype('int16')

## exploration

In [0]:
percent_missing = drug_df.isnull().sum() * 100 / len(drug_df)
missing_value_df = pd.DataFrame({'column_name': drug_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
person_id,person_id,0.000000
visit_occurrence_id,visit_occurrence_id,0.000000
year_of_birth,year_of_birth,0.000000
ethnicity_concept_id,ethnicity_concept_id,0.000000
month_of_birth,month_of_birth,0.000000
day_of_birth,day_of_birth,0.000000
race_concept_id,race_concept_id,14.292358
gender_concept_id,gender_concept_id,0.000000
birth_datetime,birth_datetime,0.000000
provider_id,provider_id,0.181530


In [0]:
drug_df = drug_df.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [0]:
drug_df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

person_id                        97916
visit_occurrence_id              94834
ethnicity_concept_id                 2
person_source_value              66223
race_concept_id                      6
gender_concept_id                    2
birth_datetime                     900
death_date                         594
death_datetime                     594
visit_start_date                   878
visit_end_date                     898
visit_start_datetime               878
visit_concept_id                     3
visit_type_concept_id                1
visit_end_datetime                 898
drug_exposure_start_date           897
drug_source_value                85293
drug_exposure_id                136275
drug_exposure_start_datetime       897
verbatim_end_date                 3098
drug_concept_id                  10736
dtype: int64

### feature engineering

In [0]:
exp = drug_df[['quantity','days_supply','age']]
exp.head(5)

,quantity,days_supply,age
4,10.0,0.0,30208
5,30.0,30.0,29726
9,30.0,30.0,26022
19,30.0,30.0,25775
21,30.0,30.0,26923


In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
### value data scaling
scaler = StandardScaler()
scaler.fit(drug_df[['quantity','days_supply','age']])
drug_df[['quantity','days_supply','age']] = scaler.transform(drug_df[['quantity','days_supply','age']])

In [0]:
def encode_onehot(df, cols):
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(orient ='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df[cols] = vec_data
    return df


In [0]:
## encoding
categorical= ['ethnicity_concept_id','race_concept_id','gender_concept_id',
             'visit_concept_id','visit_type_concept_id','drug_exposure_id','drug_concept_id']
pd.get_dummies(drug_df, columns=categorical).head(5)

In [0]:
drug_df[categorical].head(5)

In [0]:
a = [0.013689, 0.009606]
a = np.asarray(a)
t_mean = (0.013689 + 0.009606)/2
std = np.sqrt(a.var(ddof = 1))
t_stat = (a[0] - a[1])/std
t_stat 

### about measurement info

In [0]:
## for measurement value in each unit
g = sns.FacetGrid(measure_df, col='unit_concept_id')
g.map(sns.boxplot, 'death','value_as_number')

In [0]:
drug_df.loc[drug_df['value_as_number']> 100000 , 'value_as_number'] =None ##delete the extreme values

In [0]:
sns.boxplot(x = 'death', y = 'value_as_number', data = measure_df)

In [0]:
type(measure_df.measurement_datetime)

In [0]:
measure_df['measure_date']= measure_df.measurement_datetime.dt.date

In [0]:
measure_df.to_csv("measure2.csv")